In [23]:
import get_schedule
import pandas as pd
import numpy as np

In [24]:
freetime = get_schedule.Schedule().run(
        login_code="chla",
        start_date="2023-08-02",
        end_date="2023-08-07",
        names=['Benedetto, Lauren', "Freeburg, Taylor"]
    )

reading from cache


In [22]:
freetime.loc[freetime['hour'] == 23, "timestamp"] = freetime['timestamp'] + pd.Timedelta(minutes=59)
freetime['timestamp'] = freetime['timestamp'].dt.strftime("%-I:%M %p")
freetime = freetime.reset_index()
freetime_fwd = freetime.copy()
freetime_fwd['index'] = freetime_fwd['index'] + 1

ft = freetime.merge(
    freetime_fwd[['index', 'date', 'free_time']], 
    on = 'index', 
    how = 'left', 
    suffixes = ["", "_1hr_fwd"]
)

ft = ft.loc[
    (ft['date_1hr_fwd'].isnull()) 
    | ((ft['date'] == ft['date_1hr_fwd']) & (ft['free_time'] != ft['free_time_1hr_fwd']))
    | ((ft['hour'] == 23) & (ft['free_time'] == ft['free_time_1hr_fwd']))
    | (ft['hour'] == 0)
][['timestamp', 'date', 'hour', 'free_time']]

ct = 0
list_of_display_rows = []
for row in ft.to_dict("records"):
    if ct == 0 or row['hour'] == 0:
        time_start = str(row['timestamp'])
        free_at_start = row["free_time"]
        time_end = ""
        ct += 1
    elif time_end == "":
        time_end = str(row['timestamp'])
        free_at_end = row["free_time"]

        if free_at_start:
            status = "FREE"
        else:
            status = "BUSY"

        list_of_display_rows.append(
            pd.DataFrame(
                {
                    "status": [status],
                    "date": [str(row['date'])],
                    "time_period": [f"{time_start} to {time_end}"],
                }
            )
        )

        free_at_start = free_at_end
        time_start = time_end
        time_end = ""
        ct += 1
        
pd.concat(list_of_display_rows)

,status,date,time_period
0,FREE,2023-08-02,12:00 AM to 5:00 PM
0,BUSY,2023-08-02,5:00 PM to 11:59 PM
0,BUSY,2023-08-03,12:00 AM to 8:00 AM
0,FREE,2023-08-03,8:00 AM to 5:00 PM
0,BUSY,2023-08-03,5:00 PM to 11:59 PM
0,BUSY,2023-08-04,12:00 AM to 8:00 AM
0,FREE,2023-08-04,8:00 AM to 5:00 PM
0,BUSY,2023-08-04,5:00 PM to 11:59 PM
0,BUSY,2023-08-05,12:00 AM to 8:00 AM
0,FREE,2023-08-05,8:00 AM to 5:00 PM


In [26]:
df = pd.DataFrame({
    "name": ["tyler", "lauren", "taylor", "matt"]
})

In [33]:
"\r\n".join(list(df.name.unique())) + "\r\n"

'tyler\r\nlauren\r\ntaylor\r\nmatt\r\n'

In [36]:
names = list(df.name.unique())

In [39]:
string_of_names = ",".join(names)

In [42]:
list_of_files= string_of_names.split(",")
files = [(x, x) for x in list_of_files]

In [43]:
files

[('tyler', 'tyler'),
 ('lauren', 'lauren'),
 ('taylor', 'taylor'),
 ('matt', 'matt')]

In [ ]:
ft = freetime.merge(
    freetime_fwd[['index', 'date', 'free_time']], 
    on = 'index', 
    how = 'left', 
    suffixes = ["", "_1hr_fwd"]
)

ft.loc[
    (ft['date_1hr_fwd'].isnull()) 
    | ((ft['date'] == ft['date_1hr_fwd']) & (ft['free_time'] != ft['free_time_1hr_fwd']))
    | ((ft['hour'] == 23) & (ft['free_time'] == ft['free_time_1hr_fwd']))
    | (ft['hour'] == 0),
    "flag"
] = "FLAG"

In [ ]:
ft[(ft['date'].astype(str).isin(["2023-08-02", "2023-08-03"])) & (ft['flag'].notnull()) ]